# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [22]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem
from scipy import stats

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [6]:
# Run this code:
salaries = pd.read_csv('C:/Users/isabel.lopes/OneDrive - XPLOR SOLUTIONS/PythonForAISolutions/WEEK2/DAY2/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [7]:
# Your code here:

print(salaries.head())

                  Name                              Job Titles  \
0    AARON,  JEFFERY M                                SERGEANT   
1      AARON,  KARINA   POLICE OFFICER (ASSIGNED AS DETECTIVE)   
2  AARON,  KIMBERLEI R                CHIEF CONTRACT EXPEDITER   
3  ABAD JR,  VICENTE M                       CIVIL ENGINEER IV   
4    ABASCAL,  REECE E             TRAFFIC CONTROL AIDE-HOURLY   

         Department Full or Part-Time Salary or Hourly  Typical Hours  \
0            POLICE                 F           Salary            NaN   
1            POLICE                 F           Salary            NaN   
2  GENERAL SERVICES                 F           Salary            NaN   
3       WATER MGMNT                 F           Salary            NaN   
4              OEMC                 P           Hourly           20.0   

   Annual Salary  Hourly Rate  
0       101442.0          NaN  
1        94122.0          NaN  
2       101592.0          NaN  
3       110064.0          NaN  
4   

We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [8]:
# Your code here:


missing_data_values = salaries.isnull().sum()
missing_data = missing_data_values / len(salaries)*100
missing_data_output = pd.DataFrame({'Missing values': missing_data_values, 'Percentage': missing_data})
print("Missing data:")
print(missing_data_output)


Missing data:
                   Missing values  Percentage
Name                            0    0.000000
Job Titles                      0    0.000000
Department                      0    0.000000
Full or Part-Time               0    0.000000
Salary or Hourly                0    0.000000
Typical Hours               25161   75.824971
Annual Salary                8022   24.175029
Hourly Rate                 25161   75.824971


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [9]:
# Your code here:

hourly_vs_salaried_counts = salaries['Salary or Hourly'].value_counts()

print(hourly_vs_salaried_counts)


Salary or Hourly
Salary    25161
Hourly     8022
Name: count, dtype: int64


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [10]:
# Your code here:

department_counts = salaries.groupby('Department')['Name'].count()


print(department_counts)

Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [11]:
from statsmodels.stats.weightstats import ztest
# Since we have an N of 8022 and only one sample (Hourly workers), we go for a Z-test
# H0: Hourly rate == 30
# H1: Hourly rate != 30
salaries_hourly = salaries.loc[salaries['Salary or Hourly'] == 'Hourly']


# your code here

hourly_rates_workers = salaries_hourly['Hourly Rate'] #select only the "Hourly Rate" column

print(hourly_rates_workers.describe())

z_statistic, p_value = ztest(hourly_rates_workers, value=30)

print(f"Z-statistic: {z_statistic}")
print(f"P-value: {p_value}")

alpha = 0.05  # significance level
if p_value < alpha:
    print("Reject null hypothesis: The mean hourly wage is significantly different from $30/hr.")
else:
    print("Fail to reject null hypothesis: The mean hourly wage is not significantly different from $30/hr.")

count    8022.000000
mean       32.788558
std        12.112573
min         2.650000
25%        21.200000
50%        35.600000
75%        40.200000
max       109.000000
Name: Hourly Rate, dtype: float64
Z-statistic: 20.6198057854942
P-value: 1.8228873859286195e-94
Reject null hypothesis: The mean hourly wage is significantly different from $30/hr.


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [12]:
# Subset of the dataset: only Police department
salaries_police = salaries[(salaries['Department'] == 'POLICE') & (salaries['Salary or Hourly'] == 'Salary')]

In [13]:
# Since we have an N of 13404 and only one sample (salary of the police), we go for a Z-test
# H0: Annual Salary == 86000
# H1: Annual Salary > 86000

# your code here

annual_salaries_police = salaries_police['Annual Salary']

print(annual_salaries_police.describe())

z_statistic, p_value_two_tailed = ztest(annual_salaries_police, value=86000)

p_value_one_tailed = p_value_two_tailed / 2

print(f"Z-statistic: {z_statistic}")
print(f"P-value (one-tailed): {p_value_one_tailed}")

alpha = 0.05  # significance level
if p_value_one_tailed < alpha and z_statistic > 0:
    print("Reject null hypothesis: The mean annual salary is significantly greater than $86,000/year.")
else:
    print("Fail to reject null hypothesis: The mean annual salary is not significantly greater than $86,000/year.")


count     13404.000000
mean      86486.414503
std       18272.228294
min       38376.000000
25%       84054.000000
50%       90024.000000
75%       94524.000000
max      260004.000000
Name: Annual Salary, dtype: float64
Z-statistic: 3.081997005712994
P-value (one-tailed): 0.0010280845287547855
Reject null hypothesis: The mean annual salary is significantly greater than $86,000/year.


Using the `crosstab` function, find the department that has the most hourly workers. 

In [14]:
# Your code here:
department_hourly_count = pd.crosstab(index=salaries_hourly['Department'], columns='count')

max_hourly_workers_department = department_hourly_count['count'].idxmax()

max_hourly_workers_count = department_hourly_count['count'].max()

print(f"Department with the most hourly workers:", max_hourly_workers_department)
print(f"Number of people:", max_hourly_workers_count)

Department with the most hourly workers: STREETS & SAN
Number of people: 1862


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [46]:
# your code here
salaries_dept_most_h_workers =  salaries[salaries['Department'] == max_hourly_workers_department]['Hourly Rate']

# H0: The mean hourly wage is >= $35
# H1: The mean hourly wage is < $35


salaries_dept_most_h_workers = salaries_dept_most_h_workers.dropna()

print(salaries_dept_most_h_workers.isnull().sum())

print(salaries_dept_most_h_workers.mean())

t_statistic, p_value = stats.ttest_1samp(salaries_dept_most_h_workers, 35,alternative='less')

print(f"T-statistic: {t_statistic}")
print(f"P-value: {p_value}")

alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis. The hourly wage is significantly less than $35/hour.")
else:
    print("Fail to reject the null hypothesis. The hourly wage is not significantly less than $35/hour.")

0
33.72837808807734
T-statistic: -9.567447887848152
P-value: 1.6689265282353859e-21
Reject the null hypothesis. The hourly wage is significantly less than $35/hour.
